In [ ]:
'''
Analyze displacement measurement using laser displacement sensor 
Author: Yitian Shao
Created on 2022.10.17 based on 'NanoForceAnalysis'
'''

import time
from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib import mlab
import pandas as pd
import scipy.io as scio
from scipy import signal
import seaborn as sns

plt.rc('font', size=10, family='Verdana') # 'Tahoma', 'DejaVu Sans', 'Verdana'"
plt.rc('axes', edgecolor='k', linewidth=0.75, labelcolor='k')
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':True, 'top':True})
plt.rcParams['lines.linewidth'] = 1.125
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['errorbar.capsize'] = 4

figSize_inch = (3.2, 2.4)

''' Define Color Here '''
pltBlue = (32/255,120/255,180/255)
pltRed = (180/255,32/255,32/255)

''' Remove warnings '''
import warnings
warnings.filterwarnings('ignore')

In [ ]:
'''
General Functions
'''

def aPlot(figName='', is3D = False, dpi=72):
    ax = []
    
    fig1 = plt.figure(figsize = (6,3), dpi=dpi)
    
    fig1.suptitle(figName, fontsize=16)
    if(is3D):
        ax = fig1.add_subplot(111, projection='3d')
    else:
        ax = fig1.add_subplot(111)
        
    return ax, fig1

def lowpassSmooth(datain, cutFreqRatio = 0.05, order = 8):
    b, a = signal.butter(order, 2 * cutFreqRatio, btype='low')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def decodeData(fileName, numFormat, frontCode='', rearCode=''):
    segStr = re.findall(frontCode+numFormat+rearCode, fileName)
    if segStr:
        numData = float(re.findall(numFormat, segStr[0])[0])
    else:
        numData = None
    return numData

def cutRepeatTrial(rawData, Fs, expectedTrialNum, disp=False): 
    smData = lowpassSmooth(rawData, cutFreqRatio = (10/Fs), order = 8)
    
    smData = smData - smData[0]
    
#     smData = signal.detrend(smData, type='linear')
    
    samp = np.arange(len(rawData))
    
    maxRawValue = np.max(smData)
    
    ''' Fine tunning needed for individual measurement session'''
    segPointInd = np.squeeze(np.argwhere(smData[10:] > 0.25 * maxRawValue)) # Find value larger than 25% of peak as valid segment data point
    segGapInd = np.squeeze(np.argwhere(np.diff(segPointInd) > 1))# Index of point where large gap occurs (end and start of a seg)
    cutInd = (0.5 * (segPointInd[segGapInd] + segPointInd[segGapInd+1])).astype(int) # Cut in the middle of a end and a start point
    
    if not isinstance(cutInd,np.ndarray) or len(cutInd) < expectedTrialNum-1:
        if disp:
            ax0, _ = aPlot(); 
            ax0.plot(samp, rawData, color='k'); 
            plt.show();       
        return []
    
    if len(cutInd) > expectedTrialNum-1:
        cutInd = cutInd[:expectedTrialNum]
    elif len(cutInd) == expectedTrialNum-1:
        cutInd = np.insert(cutInd, expectedTrialNum-1, len(rawData)-1)

    avgSegLen = np.mean(np.diff(cutInd))
    cutInd = np.insert(cutInd, 0, max(cutInd[0]-avgSegLen, 0))
    
    if disp:
        ax0, _ = aPlot(); 
        ax0.plot(samp, rawData, color='k'); 
        ax0.plot(samp, smData, color='tab:blue')
        ax0.plot(cutInd, np.zeros(cutInd.shape), '*r')
        plt.show();
    
    return cutInd

def getDataArray(dFrame, indList, colName, toFlat=False):
    compValue = (dFrame.loc[indList,colName]).to_numpy()  
    
    if toFlat:
        flatValue = []
        for aRow in compValue:
            flatValue.extend(aRow)
        flatValue = np.array(flatValue)
        return flatValue
            
    return compValue
    
def getMeanSTD(compValue): # Compute errorbar of data mixing all conditions (rows of input array)
    if len(compValue) == 0:
        return None, None
        
    compValue = np.array(compValue)
    
    meanValue = np.mean(compValue)
    stdValue = np.std(compValue)
    
    return meanValue, stdValue

In [ ]:
'''
Load, preprocess and segment displacement data
'''

# measureDataPath = './DispData20221017Voltage_old60mmand110mm'
measureDataPath = './DispData20221212Voltage'

# measureDataPath = './DispData20221017Frequency' # 110mm
# measureDataPath = './DispData20221212Frequency' # 60mm and 110mm

Fs = None 

laserData = []

''' Data preprocess and segmentation '''
for root, directories, files in walk(measureDataPath):
    for fileName in files:
        measureDate = decodeData(fileName, '\d{4}', rearCode='2022')
        tubeLen = decodeData(fileName, '[\d+\.]*\d+', rearCode='mm')
        infillVol = decodeData(fileName, '[\d+\.]*\d+', rearCode='mL') 
        pulseFreq = decodeData(fileName, '[\d+\.]*\d+', rearCode='mHz')  
        
        sinFreq = decodeData(fileName, '[\d+\.]*\d+', rearCode='Hz') 
        voltage = decodeData(fileName, '[\d+\.]*\d+', rearCode='kV') 

        trialNum = decodeData(fileName, '\d+', rearCode='.txt') # DispData20221017
        if trialNum is None:
            trialNum = 1

        dLabel = ""
        if tubeLen is not None:
            dLabel = dLabel + ("%.1fmm" % tubeLen)    
        if infillVol is not None:
            dLabel = dLabel + ("%.2fmL" % infillVol)           
        if sinFreq is not None:
            dLabel = dLabel + ("%.1fHz" % sinFreq)
        if voltage is not None:
            dLabel = dLabel + ("%.1fkV" % voltage)

        print("%s - trial%s" % (dLabel, trialNum))
    
        readData = pd.read_csv(ospa.join(root, fileName), header = None)
        
        if readData is not None:       
            t = readData[0].to_numpy()
            diffT = np.diff(t)
            diffT = diffT[diffT > 0]
            SampleRate = int(1.0/np.mean(diffT))
            if SampleRate != Fs:
                if Fs is None:
                    Fs = SampleRate
                    print("Sampling frequenecy = %.2f Hz" % Fs)
                else:
                    print("Sampling frequenecy of current data %.2fHz differs from the Fs = %.2f Hz" % (SampleRate, Fs))
                    
            v = readData[1].to_numpy()
            dispData = readData[2].to_numpy()
            
            data = []
            
            if re.findall('Frequency', measureDataPath): # No data segmentation is needed for frequency response test
                data.append(dispData)  
            else:
                cutInd = cutRepeatTrial(dispData, Fs, 6, disp=True)
                segNum = len(cutInd)-1

                if segNum < 0:
                    cutInd = cutRepeatTrial(dispData, Fs, 5, disp=True)
                    segNum = len(cutInd)-1

                if segNum > 0: 
                    for j in range(segNum):
                        segData = dispData[cutInd[j]:cutInd[j+1]]
                        data.append(segData)  
                    
            laserData.append([dLabel, tubeLen, sinFreq, voltage, trialNum, data])

laserData = pd.DataFrame(laserData, columns = ['Label', 'TubeLength_mm', 'SinFreq_Hz', 'Voltage_kV','Trial','Data'])  

laserData.tail(20)


In [ ]:
'''(Exclusive) Frequency response characteristics for the actuator'''

labels = laserData.Label.unique()

# labels = ['60.0mm0.4Hz7.0kV','60.0mm0.6Hz7.0kV', '60.0mm1.6Hz7.0kV', '60.0mm10.0Hz7.0kV',
#  '60.0mm12.6Hz7.0kV', '60.0mm1.0Hz7.0kV', '60.0mm2.5Hz7.0kV',
#  '60.0mm2.0Hz7.0kV', '60.0mm3.2Hz7.0kV', '60.0mm4.0Hz7.0kV',
#  '60.0mm5.0Hz7.0kV', '60.0mm6.3Hz7.0kV', '60.0mm7.9Hz7.0kV']; # Fs = 98
# print(labels) 

dataFeature = []

concateSignal = np.empty([0, 1], dtype=float)

for aLabel in labels:
    selectedData = laserData.loc[(laserData['Label'] == aLabel)]
    
    freqMag = []
    
    for index, row in selectedData.iterrows():
        for aSignal in row["Data"]:           
            y = aSignal
            y = y - np.amin(y)
            
            # data processing to remove the inital overshooting
            truncStart = np.argmax(y > (0.5*np.max(y)))+500
            truncEnd = int(10*Fs) + truncStart # Truncate only 10 seconds of measurement
            y = y[truncStart:truncEnd]
            
            RMSdispl= 2*np.sqrt(np.mean((y - np.mean(y))**2))
            
            concateSignal = np.append(concateSignal, y)
            
            _, ax1 = plt.subplots(1,2, dpi=72, figsize=(12,2))
            ax1[0].plot((np.arange(len(y))/Fs),y, 'k')
            ax1[0].set_xlabel('Time (s)')
            ax1[0].set_ylabel('Displacement (mm)')
            ax1[0].set_title('RMS Displ. = %.6f mm' % RMSdispl)
            
            spectr, f, _ = ax1[1].magnitude_spectrum(y, Fs=Fs, color='C1', window=mlab.window_none)
            
            ind = np.argwhere(f < 0.3)
            spectr[ind] = 0
            
            idx = spectr.argmax()           
            
            freqMagValue = spectr[idx]
            
            actualPeakFreq = f[idx]
            
            print("Desired sin frequency = %.2f Hz - Actually frequency = %.2f Hz" % (row['SinFreq_Hz'], actualPeakFreq))
            
            if abs(row['SinFreq_Hz'] - actualPeakFreq) > 10:
                print("failed")
                freqMagValue = np.float64(1e-9)

            ax1[1].plot(actualPeakFreq, freqMagValue, '.b')    
            ax1[1].set_xlim([0.1, Fs/2])
            ax1[1].set_ylabel("Magnitude")
            ax1[1].set_title(row['Label'])
            
            freqMag.append(freqMagValue)
            
    dataFeature.append([aLabel, row["TubeLength_mm"], row['SinFreq_Hz'], freqMag])

dataFeature = pd.DataFrame(dataFeature, columns = ['Label', 'TubeLength_mm','SinFreq_Hz','FreqMag'])  

dataFeature.tail(20)

In [ ]:
'''(Exclusive) Produce frequency response plot'''
def freqResponse(dframe, dispFrequencies, yColName, yColSelect):
    dispFreqMag = []
    for aFreq in dispFrequencies:
        freqMag = dframe.loc[(dframe['SinFreq_Hz'] == aFreq) & (dframe[yColName] == yColSelect), 'FreqMag']
        if len(freqMag) == 0:
            print("Failed: %f Hz" % aFreq)
            dispFreqMag.append(1e-9)
        else:
            freqMag = freqMag.values[0][0].tolist()
            dispFreqMag.append(freqMag)

    dispFreqMag = np.array(dispFreqMag)

    dispLogMag = 20*np.log10(dispFreqMag)
    
    logMagZero = 20*np.log10(0.25) # Define 1mm peak-to-peak displacement as 0dB
    
    dispLogMag = dispLogMag-logMagZero
    
    return dispLogMag, dispFreqMag * 4 # (dB, mm)

octaveFrequencies = [4.000e-01, 6.000e-01, 1.000e+00, 1.600e+00, 2.000e+00, 2.500e+00, 3.200e+00,
                     4.000e+00, 5.000e+00, 6.300e+00, 7.900e+00, 1.000e+01, 1.260e+01, 1.580e+01,
                     2.000e+01, 2.510e+01, 3.160e+01, 3.98e+01, 5.010e+01, 6.310e+01, 7.940e+01,
                     1.000e+02, 1.259e+02, 1.585e+02, 1.995e+02, 2.512e+02, 3.162e+02, 3.981e+02,
                     5.012e+02] 

dispFrequencies = [0.4, 1, 2, 4, 10, 20, 40, 100, 200, 500]

dispLogMag110, mag110 = freqResponse(dataFeature, octaveFrequencies, 'TubeLength_mm', 110)
x110 = np.log10(octaveFrequencies)

dispLogMag60,_ = freqResponse(dataFeature, octaveFrequencies, 'TubeLength_mm', 60)

# avgFreqMag = mag110
# print("\n ----- Mixed data for Frequency response ----- ")
# print("Magnitude (dB)")
# print(dispLogMag110)
# print("Absolute force magnitude (mm)")
# print(avgFreqMag)
# print("Mean - 0.4 Hz to 10 Hz = %.1f mm" % np.mean(avgFreqMag[:12]))
# print("Mean - 10 Hz to 40 Hz = %.1f mm" % np.mean(avgFreqMag[12:18]))
# print("Mean - 40 Hz to 500 Hz = %.1f mm" % np.mean(avgFreqMag[18:]))
# print("Frequency (Hz)")
# print(octaveFrequencies)

xTicks = np.log10(dispFrequencies)
xLabel = 'Frequency (Hz)'

colors = sns.color_palette('hls', n_colors=3)
# colors = sns.light_palette(pltRed, reverse=True, n_colors=(5+8))

fig1, ax1 = plt.subplots(dpi=300, figsize=(5,1))

ax1.plot([x110[0], x110[-1]], [0, 0], '--', c='0.5')

ax1.plot(x110, dispLogMag60, '.-', c=colors[1])
ax1.plot(x110, dispLogMag110, '.-', c=colors[0])
ax1.set_ylabel('Magnitude (dB)')

ax1.set_xlabel(xLabel)
ax1.set_xticks(xTicks);
ax1.set_xticklabels(dispFrequencies);

ax1.legend(labels=['1.0mm', '60mm', '110mm'], frameon=False, bbox_to_anchor=(1.0, 0.9), loc='upper left');
ax1.set_yticks([-40, -20, 0])
ax1.set_ylim([-42, 22])

fig1.savefig("freq_response_displ_60and110.pdf", bbox_inches='tight')

In [ ]:
'''
Pulse train measurements:

Feature extraction from signals: peak value, time-averaged value, rising time
'''

labels = laserData.Label.unique()
print(labels)

dataFeature = []

for aLabel in labels:
    selectedData = laserData.loc[(laserData['Label'] == aLabel)]

    peakF = []
    rawPeakF = []
    tavgF = []
    triseF = []
    smF = []
        
    for index, row in selectedData.iterrows():
        for aSignal in row["Data"]:
            smData = lowpassSmooth(aSignal, cutFreqRatio = (20/Fs), order = 10)
            smData = smData - smData[0]

            maxValue = np.max(smData)
            onsetInd = np.argwhere(smData[50:] > 0.20 * maxValue)[0][0]+50 # When signal reach 10% of its peak (onset)

            PP80Ind = np.argwhere(smData[50:] > 0.80 * maxValue)[0][0]+50 # When signal reach 90% of its peak

            peakInd = np.argmax(smData)
            peakValue = np.max(smData)           
            rawPeakValue = np.max(aSignal[:peakInd]-aSignal[0])

            riseTime = (PP80Ind - onsetInd)/Fs
            
            tavgValue = np.mean(smData[onsetInd:PP80Ind]) 
            
            peakF.append(peakValue)
            rawPeakF.append(rawPeakValue)
            tavgF.append(tavgValue)
            triseF.append(riseTime)
            smF.append(smData)
            
            ax1,_ = aPlot(dpi=72)
            ax1.plot(aSignal-aSignal[0], 'k')
#             ax1.plot([1, peakInd], [peakValue, peakValue], '--b')
            ax1.plot([1, peakInd], [rawPeakValue, rawPeakValue], '--c')
            ax1.plot(smData, 'g')
            ax1.plot([onsetInd, onsetInd], [0, maxValue], '--r')
            ax1.plot([PP80Ind, PP80Ind], [0, maxValue], '--r')
            ax1.set_xlabel("%s: peak=%.2fN , tAvg=%.2fN , riseT=%.3fs" % (aLabel, peakValue,tavgValue,riseTime))
            ax1.set_xlim([onsetInd-100, peakInd + 400])
            ax1.set_ylabel("Displacement (mm)")
            plt.show()
            
    dataFeature.append([aLabel, row["TubeLength_mm"], row["Voltage_kV"], peakF, rawPeakF, tavgF, triseF, smF])

dataFeature = pd.DataFrame(dataFeature, columns = ['Label', 'TubeLength_mm','Voltage_kV','PeakForce','RawPeakForce','AvgForce','TriseForce','Data'])  

dataFeature.head(10)

In [ ]:
'''
Functions for Computing and Visualizing Errorbar
'''
def computeErrorbar1Var(dframe, xColName, xColList, zColName):
    dMean = []
    dSTD = []
    
    for x in xColList:
        selectInd = (dframe[xColName] == x)

        compValue = getDataArray(dframe, selectInd, zColName, toFlat=True)  

        valueMean, valueSTD = getMeanSTD(compValue)

        dMean.append(valueMean)
        dSTD.append(valueSTD)
    
    dMean = np.array(dMean)
    dSTD = np.array(dSTD)
    
    return dMean, dSTD

def computeErrorbar(dframe, xColName, xColList, yColName, yColList, zColName):
    dMean = []
    dSTD = []
    
    for x in xColList:
        dMean.append([])
        dSTD.append([])
        for y in yColList:
            selectInd = (dframe[xColName] == x) & (dframe[yColName] == y)

            compValue = getDataArray(dframe, selectInd, zColName, toFlat=True)  
            
            valueMean, valueSTD = getMeanSTD(compValue)
            
            dMean[-1].append(valueMean)
            dSTD[-1].append(valueSTD)
    
    dMean = np.array(dMean)
    dSTD = np.array(dSTD)
    
    return dMean, dSTD

def dispErrorbar(ax, x, dMean, dErr, xLabel, xLabelStr, yLabelStr, zLabel, colors=['C1'], style='.-', 
                 dispLegend=True, xShift=[0.1,0.4]):
    yLen = len(yLabelStr)
    
    ax.set_ylabel(zLabel, color='k')
    ax.tick_params(axis='y', labelcolor='k')
    ax.spines['bottom'].set_color('k')
    ax.spines['top'].set_color('k') 
    ax.spines['right'].set_color('k')
    ax.spines['left'].set_color('k')

    pltHandles = []
    for i in np.arange(yLen):
        pltHandles.append(ax.plot(2*x + (i * xShift[0]), dMean[i], style, c=colors[i])[0])
        (ax.errorbar(2*x + (i * xShift[0]), dMean[i], yerr=dErr[i], fmt='none', ecolor=colors[i], capsize=0)[0]) 
        
    ax.set_xlabel(xLabel)
    ax.set_xticklabels(xLabelStr, rotation=0);
    ax.set_xticks(2*x + xShift[1]);

    if dispLegend:
        ax.legend(handles=reversed(pltHandles), labels=reversed(yLabelStr), frameon=False, bbox_to_anchor=(1.02, 1.0), 
                  loc='upper left');
    
    return ax

In [ ]:
'''
Analysis for Visualizing the Errorbar of multiple actuator types
'''
# labels = ['60mm3.0kV', '60mm4.0kV', '60mm5.0kV', '60mm6.0kV', '60mm7.0kV',
#           '110mm3.0kV', '110mm4.0kV', '110mm5.0kV', '110mm6.0kV', '110mm7.0kV']

vLevels = [3, 4, 5, 6, 7]
tLengths = [60, 110]

colors = sns.color_palette('hls', n_colors=3)
style = '.-'

xLen = len(vLevels)
x = np.arange(xLen)
xLabelStr = np.array(vLevels)
yLabelStr = tLengths
xLabel = 'Voltage (kV)'


meanPeakPdc, stdPeakPdc = computeErrorbar(dataFeature, 'TubeLength_mm', tLengths, 'Voltage_kV', vLevels, 'PeakForce')   
meanRawPeakF, stdRawPeakF = computeErrorbar(dataFeature, 'TubeLength_mm', tLengths, 'Voltage_kV', vLevels, 'RawPeakForce')

print("Raw Displacement - Mean (n=12, 60mm/110mm)")
print("60mm: %s \n110mm: %s" % (meanRawPeakF[0], meanRawPeakF[1]))
print("Raw Displacement - STD (n=12, 60mm/110mm)")
print("60mm: %s \n110mm: %s" % (stdRawPeakF[0], stdRawPeakF[1]))

fig1, ax1 = plt.subplots(dpi=300, figsize=figSize_inch)

dispErrorbar(ax1, x, meanRawPeakF, stdRawPeakF, xLabel, xLabelStr, yLabelStr, 'Displacement (mm)', colors, style=style,
            xShift=[0.1,0.05])

ax1.set_ylim([0, 6])

# fig1.savefig("displace_voltage_60110.pdf", bbox_inches='tight')


'''Average of two actuators'''
meanRawPeakF, stdRawPeakF = computeErrorbar1Var(dataFeature, 'Voltage_kV', vLevels, 'RawPeakForce')

print("\n ----- Mixed data  ----- ")
print("Raw Displacement - Mean (n=12, 60mm + 110mm):")
print(meanRawPeakF)
print("Raw Displacement - STD (n=12, 60mm + 110mm):")
print(stdRawPeakF)

fig2, ax2 = plt.subplots(dpi=300, figsize=(1,2))    

colors = sns.light_palette(pltRed, reverse=True, n_colors=(xLen+8))

for i in range(xLen):
    ax2.plot(vLevels[i], meanRawPeakF[i], '.', c=colors[xLen-i])
    ax2.errorbar(vLevels[i], meanRawPeakF[i], yerr=stdRawPeakF[i], fmt='none', ecolor=colors[xLen-i], capsize=0)

ax2.set_ylabel('Displacement (mm)')
ax2.set_xlabel(xLabel)
ax2.set_xticks(vLevels);

ax2.set_xlim([vLevels[0]-0.5, vLevels[-1]+0.5])
ax2.set_ylim([0, 6])

# fig2.savefig("displace_voltage_avg.pdf", bbox_inches='tight')

In [ ]:
'''Plot signal waveform of sinusoidal voltage'''
aSignal = laserData.loc[(laserData['Label'] == '110.0mm1.0Hz7.0kV'), 'Data'].values[0][0]
aSignal2 = laserData.loc[(laserData['Label'] == '110.0mm39.8Hz7.0kV'), 'Data'].values[0][0]

aSignal = aSignal - np.amin(aSignal)
aSignal2 = aSignal2 - np.mean(aSignal2)

fig1, ax = plt.subplots(2,1,dpi=300, figsize=(6,2))
ax[0].plot((np.arange(aSignal.shape[0])/Fs - 0.47) * 1000, aSignal, 'C1')
# ax[0].set_xlim([0, 10000])
# ax[0].set_ylim([0, 4])
ax[0].set_ylabel('Displacement (mm)')

ax[1].plot((np.arange(aSignal2.shape[0])/Fs) * 1000, aSignal2, 'C2')
# ax[1].set_xlim([1000, 2000])
# ax[1].set_ylim([0.1, 0.15])
ax[1].set_xlabel('Time (ms)')

fig1.savefig("displ_time_sin1sin40.pdf", bbox_inches='tight')

In [ ]:
'''Plot signal waveform of DC voltage'''
aSignal = laserData.loc[(laserData['Label'] == '110.0mm7.0kV'), 'Data'].values[0][0]
aSignal2 = laserData.loc[(laserData['Label'] == '110.0mm3.0kV'), 'Data'].values[0][0]

fig1, ax1 = plt.subplots(dpi=300, figsize=(5,2))

ax1.plot(np.arange(aSignal.shape[0])/Fs, aSignal-aSignal[0], c=colors[0])
ax1.plot(np.arange(aSignal2.shape[0])/Fs-1.3, aSignal2-aSignal2[0], c=colors[xLen])

ax1.set_xlim([0.4, 1.20])
ax1.set_ylim([-0.2, 5])
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Displacement (mm)')

ax1.legend(labels=['7kV','3kV'], frameon=False, bbox_to_anchor=(1.0, 0.9), loc='upper left');

fig1.savefig("displ_time_7kv3kv.pdf", bbox_inches='tight')